In [2]:
import os
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader

loaded_document = TextLoader('./data/state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)

chunks_of_text = text_splitter.split_documents(loaded_document)

embeddings = OpenAIEmbeddings()

faiss_db = FAISS.from_documents(chunks_of_text,embeddings)

retriever = faiss_db.as_retriever(search_kwargs={"k":3})



### Using LCEL

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [5]:
template = """ Answer the question based only on the following context:

{context}

Question : {question}

"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [7]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model 
    | StrOutputParser()
)

chain.invoke("what did he say about ketanji brown jackson?")

"He mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson and described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He also mentioned that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers. Additionally, he highlighted the broad range of support she has received since being nominated."